In [22]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

import eda
from utils import get_categorical_features, get_numeric_features, parallel_process
from feature_engineering import base_aggregation, diff_feature, division_feature, product_feature, cnt_encoding, select_category_value_agg, exclude_feature, target_encoding
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

base = utils.read_df_pkl('../input/base0*')[[key, target]].set_index(key)

100%|██████████| 3/3 [00:00<00:00, 582.11it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
"""
PREVIOUS
"""
'''
#========================================================================
# PREVIOUS
Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64
TUESDAY      223438
WEDNESDAY    223097
MONDAY       222834
FRIDAY       221579
THURSDAY     218463
SATURDAY     216460
SUNDAY       151179
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64
Y    1477046
N          4
Name: FLAG_LAST_APPL_PER_CONTRACT, dtype: int64
XAP                                 729497
XNA                                 677918
Repairs                              23765
Other                                15608
Urgent needs                          8412
Buying a used car                     2888
Building a house or an annex          2693
Everyday expenses                     2416
Medicine                              2174
Payments on other loans               1931
Education                             1573
Journey                               1239
Purchase of electronic equipment      1061
Buying a new car                      1012
Wedding / gift / holiday               962
Buying a home                          865
Car repairs                            797
Furniture                              749
Buying a holiday home / land           533
Business development                   426
Gasification / water supply            300
Buying a garage                        136
Hobby                                   55
Money for a third person                25
Refusal to name the goal                15
Name: NAME_CASH_LOAN_PURPOSE, dtype: int64
Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64
Cash through the bank                        1033552
XNA                                           434220
Non-cash from your account                      8193
Cashless from the account of the employer       1085
Name: NAME_PAYMENT_TYPE, dtype: int64
XAP       1209467
HC         142778
LIMIT       50479
SCO         34538
CLIENT      26431
SCOFR        6109
XNA          3836
VERIF        3329
SYSTEM         83
Name: CODE_REJECT_REASON, dtype: int64
XNA                876255
Unaccompanied      466732
Family             203549
Spouse, partner     64734
Children            30819
Other_B             17116
Other_A              8836
Group of people      2173
Name: NAME_TYPE_SUITE, dtype: int64
Repeater     1064852
New           290678
Refreshed     119795
XNA             1725
Name: NAME_CLIENT_TYPE, dtype: int64
XNA                         758131
Mobile                      224686
Consumer Electronics        121237
Computers                   105748
Audio/Video                  99357
Furniture                    53640
Photo / Cinema Equipment     25018
Construction Materials       24995
Clothing and Accessories     23554
Auto Accessories              7381
Jewelry                       6290
Homewares                     5023
Medical Supplies              3843
Vehicles                      3370
Sport and Leisure             2981
Gardening                     2668
Other                         2553
Office Appliances             2333
Tourism                       1659
Medicine                      1550
Direct Sales                   446
Fitness                        209
Additional Service             128
Education                      107
Weapon                          77
Insurance                       64
House Construction               1
Animals                          1
Name: NAME_GOODS_CATEGORY, dtype: int64
POS     691011
Cash    461563
XNA     324051
Cars       425
Name: NAME_PORTFOLIO, dtype: int64
XNA        1015487
x-sell      361984
walk-in      99579
Name: NAME_PRODUCT_TYPE, dtype: int64
Credit and cash offices       594158
Country-wide                  467925
Stone                         206268
Regional / Local              102602
Contact center                 59987
AP+ (Cash loan)                40904
Channel of corporate sales      4756
Car dealer                       450
Name: CHANNEL_TYPE, dtype: int64
XNA                     701189
Consumer electronics    374400
Connectivity            263933
Furniture                56385
Construction             29441
Clothing                 23462
Industry                 19086
Auto technology           4854
Jewelry                   2668
MLM partners              1121
Tourism                    511
Name: NAME_SELLER_INDUSTRY, dtype: int64
middle        385532
high          353331
XNA           324051
low_normal    322095
low_action     92041
Name: NAME_YIELD_GROUP, dtype: int64
Cash                              285990
POS household with interest       263622
POS mobile with interest          220670
XNA                               193510
Cash X-Sell: middle               143883
Cash X-Sell: low                  130248
POS industry with interest         98833
POS household without interest     82908
Cash Street: high                  59639
Cash X-Sell: high                  59301
Cash Street: middle                34658
Cash Street: low                   33834
POS mobile without interest        24082
POS other with interest            23879
POS industry without interest      12602
POS others without interest         2555
Name: PRODUCT_COMBINATION, dtype: int64
WEDNESDAY    31913
TUESDAY      31680
MONDAY       30723
THURSDAY     30636
FRIDAY       30469
SATURDAY     24171
SUNDAY       13572
Name: revo_WEEKDAY_APPR_PROCESS_START, dtype: int64
Y    184693
N      8471
Name: revo_FLAG_LAST_APPL_PER_CONTRACT, dtype: int64
XAP    193164
Name: revo_NAME_CASH_LOAN_PURPOSE, dtype: int64
XNA    193164
Name: revo_NAME_PAYMENT_TYPE, dtype: int64
XAP       143626
HC         32453
SCOFR       6702
LIMIT       5201
SCO         2929
XNA         1408
SYSTEM       634
VERIF        206
CLIENT         5
Name: revo_CODE_REJECT_REASON, dtype: int64
Unaccompanied      42238
Family              9714
Spouse, partner     2335
Children             747
Other_B              508
Other_A              241
Group of people       67
Name: revo_NAME_TYPE_SUITE, dtype: int64
Repeater     166409
Refreshed     15854
New           10685
XNA             216
Name: revo_NAME_CLIENT_TYPE, dtype: int64
XNA                         192678
Consumer Electronics           339
Audio/Video                     84
Mobile                          22
Computers                       21
Furniture                       16
Photo / Cinema Equipment         3
Other                            1
Name: revo_NAME_GOODS_CATEGORY, dtype: int64
Cards    144985
XNA       48179
Name: revo_NAME_PORTFOLIO, dtype: int64
x-sell     94303
walk-in    50682
XNA        48179
Name: revo_NAME_PRODUCT_TYPE, dtype: int64
Credit and cash offices       125810
Country-wide                   26765
AP+ (Cash loan)                16142
Contact center                 11310
Regional / Local                5926
Stone                           5815
Channel of corporate sales      1394
Car dealer                         2
Name: revo_CHANNEL_TYPE, dtype: int64
XNA                     154531
Consumer electronics     23865
Connectivity             12096
Furniture                 1464
Clothing                   487
Construction               340
Auto technology            136
Industry                   108
MLM partners                94
Jewelry                     41
Tourism                      2
Name: revo_NAME_SELLER_INDUSTRY, dtype: int64
XNA    193164
Name: revo_NAME_YIELD_GROUP, dtype: int64
Card Street    112582
Card X-Sell     80582
Name: revo_PRODUCT_COMBINATION, dtype: int64
#========================================================================
'''

#========================================================================
# Data Load 
pre = utils.read_df_pkl(path='../input/clean_pre*.p')
display(pre.columns)

ignore_list = [key, target, 'SK_ID_PREV']
loan = pre[pre['NAME_CONTRACT_TYPE']!='Revolving loans']
revo = pre[pre['NAME_CONTRACT_TYPE']=='Revolving loans']

num_list = get_numeric_features(df=revo, ignore_list=ignore_list)

revo.columns = [f"revo_{col}" if col in num_list else col for col in revo.columns]

pre = pd.concat([loan, revo], axis=0)
org_col_list = pre.columns.tolist()
#========================================================================

100%|██████████| 3/3 [00:00<00:00,  3.71it/s]


Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [24]:
#========================================================================
# Simplete Aggregation
#========================================================================
prefix_list = ['', 'revo_']
df = pre

for prefix in prefix_list:
    
    amt_cre = f'{prefix}AMT_CREDIT'
    amt_ann = f'{prefix}AMT_ANNUITY'
    amt_down = f'{prefix}AMT_DOWN_PAYMENT'
    amt_app = f'{prefix}AMT_APPLICATION'
    amt_good = f'{prefix}AMT_GOODS_PRICE'
    rate_down = f'{prefix}RATE_DOWN_PAYMENT'
    
    combi_list = [
        [amt_cre, amt_ann]
        ,[amt_cre, amt_down]
        ,[amt_good, amt_down]
        ,[amt_app, amt_down]
        ,[amt_down, amt_ann]
    ]

    
    for (col1, col2) in combi_list:
        df[f"{col1}_per_{col2}"] = df[col1] / (df[col2]+1)
    
    # ざっくり金利
    df[f'{prefix}TMP_INTEREST_RATE_APP'] = (df[amt_cre] - df[amt_app]) / (df[amt_app]+1)
    df[f'{prefix}TMP_INTEREST_RATE_GOODS'] = (df[amt_cre] - df[amt_good]) / (df[amt_good]+1)
    df[f'{prefix}AMT_INTEREST'] = df[amt_cre] - df[amt_app]
    df[f'{prefix}AMT_INTEREST_GOODS'] = df[amt_cre] - df[amt_good]
    
    df[f'{prefix}CNT_CREDIT_diff_CNT_APP'] = (df[amt_cre] - df[amt_ann])- (df[amt_app] - df[amt_ann])
    df[f'{prefix}CNT_CREDIT_diff_CNT_GOODS'] = (df[amt_cre] - df[amt_ann])- (df[amt_good] - df[amt_ann])
    df[f"{prefix}DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN"] = (df[amt_cre] * df[rate_down]) - df[amt_down]
    
    # DAYS FIRST
    f_draw = f'{prefix}DAYS_FIRST_DRAWING'
    dec = f'{prefix}DAYS_DECISION'
    f_due = f'{prefix}DAYS_FIRST_DUE'
    l_due = f'{prefix}DAYS_LAST_DUE'
    l_due1 = f'{prefix}DAYS_LAST_DUE_1ST_VERSION'
    terminate = f'{prefix}DAYS_TERMINATION'
    
    df[f'{prefix}FIRST_DRAW_TERM'] = df[f_draw] - df[dec]
    df[f'{prefix}FIRST_DRAW_DUE_TERM'] = df[f_draw] - df[f_due]
    df[f'{prefix}FIRST_DUE_TERM'] = df[f_due] - df[dec]
    
    # DAYS TERM 
    df[f'{prefix}DAYS_LAST_DUE_diff_FIRST_DUE'] = df[l_due] - df[f_due]
    df[f'{prefix}DAYS_LAST_DUE_1st_diff_FIRST_DUE'] = df[l_due1] - df[f_due]
    df[f'{prefix}DAYS_LAST_DUE_diff_DECISION'] = df[l_due] - df[dec]
    df[f'{prefix}DAYS_LAST_DUE_1st_diff_DECISION'] = df[l_due1] - df[dec]
    df[f'{prefix}DAYS_TERMINATION_diff_DECISION'] = df[terminate] - df[dec]
    
display(df.head())

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_TERMINATION,FLAG_LAST_APPL_PER_CONTRACT,HOUR_APPR_PROCESS_START,NAME_CASH_LOAN_PURPOSE,NAME_CLIENT_TYPE,NAME_CONTRACT_STATUS,NAME_CONTRACT_TYPE,NAME_GOODS_CATEGORY,NAME_PAYMENT_TYPE,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,NAME_SELLER_INDUSTRY,NAME_TYPE_SUITE,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_CURR,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START,revo_AMT_ANNUITY,revo_AMT_APPLICATION,revo_AMT_CREDIT,revo_AMT_DOWN_PAYMENT,revo_AMT_GOODS_PRICE,revo_CNT_PAYMENT,revo_DAYS_DECISION,revo_DAYS_FIRST_DRAWING,revo_DAYS_FIRST_DUE,revo_DAYS_LAST_DUE,revo_DAYS_LAST_DUE_1ST_VERSION,revo_DAYS_TERMINATION,revo_HOUR_APPR_PROCESS_START,revo_NFLAG_INSURED_ON_APPROVAL,revo_NFLAG_LAST_APPL_IN_DAY,revo_RATE_DOWN_PAYMENT,revo_RATE_INTEREST_PRIMARY,revo_RATE_INTEREST_PRIVILEGED,revo_SELLERPLACE_AREA,AMT_CREDIT_per_AMT_ANNUITY,AMT_CREDIT_per_AMT_DOWN_PAYMENT,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT,AMT_APPLICATION_per_AMT_DOWN_PAYMENT,AMT_DOWN_PAYMENT_per_AMT_ANNUITY,TMP_INTEREST_RATE_APP,TMP_INTEREST_RATE_GOODS,AMT_INTEREST,AMT_INTEREST_GOODS,CNT_CREDIT_diff_CNT_APP,CNT_CREDIT_diff_CNT_GOODS,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN,FIRST_DRAW_TERM,FIRST_DRAW_DUE_TERM,FIRST_DUE_TERM,DAYS_LAST_DUE_diff_FIRST_DUE,DAYS_LAST_DUE_1st_diff_FIRST_DUE,DAYS_LAST_DUE_diff_DECISION,DAYS_LAST_DUE_1st_diff_DECISION,DAYS_TERMINATION_diff_DECISION,revo_AMT_CREDIT_per_revo_AMT_ANNUITY,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT,revo_AMT_APPLICATION_per_revo_AMT_DOWN_PAYMENT,revo_AMT_DOWN_PAYMENT_per_revo_AMT_ANNUITY,revo_TMP_INTEREST_RATE_APP,revo_TMP_INTEREST_RATE_GOODS,revo_AMT_INTEREST,revo_AMT_INTEREST_GOODS,revo_CNT_CREDIT_diff_CNT_APP,revo_CNT_CREDIT_diff_CNT_GOODS,revo_DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN,revo_FIRST_DRAW_TERM,revo_FIRST_DRAW_DUE_TERM,revo_FIRST_DUE_TERM,revo_DAYS_LAST_DUE_diff_FIRST_DUE,revo_DAYS_LAST_DUE_1st_diff_FIRST_DUE,revo_DAYS_LAST_DUE_diff_DECISION,revo_DAYS_LAST_DUE_1st_diff_DECISION,revo_DAYS_TERMINATION_diff_DECISION
0,1730.430,17145.0,17145.0,NaN,17145.0,Country-wide,12.0,XAP,-73.0,NaN,-42.0,-42.0,300.0,-37.0,Y,15.0,XAP,Repeater,Approved,Consumer loans,Mobile,Cash through the bank,POS,XNA,Connectivity,XNA,middle,0.0,1.0,POS mobile with interest,NaN,0.182832,0.867336,35.0,271877,2030495,SATURDAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.902220,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,31.0,0.0,342.0,31.0,373.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25188.615,607500.0,679671.0,NaN,607500.0,Contact center,36.0,XAP,-164.0,NaN,-134.0,NaN,916.0,NaN,Y,11.0,XNA,Repeater,Approved,Cash loans,XNA,XNA,Cash,x-sell,XNA,Unaccompanied,low_action,1.0,1.0,Cash X-Sell: low,NaN,NaN,NaN,-1.0,108129,2802425,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.982191,NaN,NaN,NaN,NaN,0.118800,0.118800,72171.0,72171.0,72171.0,72171.0,NaN,NaN,NaN,30.0,NaN,1050.0,NaN,1080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15060.735,112500.0,136444.5,NaN,112500.0,Credit and cash offices,12.0,XAP,-301.0,NaN,-271.0,NaN,59.0,NaN,Y,11.0,XNA,Repeater,Approved,Cash loans,XNA,Cash through the bank,Cash,x-sell,XNA,"Spouse, partner",high,1.0,1.0,Cash X-Sell: high,NaN,NaN,NaN,-1.0,122040,2523466,TUESDAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.059016,NaN,NaN,NaN,NaN,0.212838,0.212838,23944.5,23944.5,23944.5,23944.5,NaN,NaN,NaN,30.0,NaN,330.0,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,47041.335,450000.0,470790.0,NaN,450000.0,Credit and cash off

In [25]:
prefix = 'f002_pre_'
aggs = {}

for col in df.columns:
    if col in org_col_list:
        continue
    aggs[col] = ['sum', 'mean', 'max', 'min', 'std']
    
    
df_agg = df.groupby(key).agg(aggs)
new_col_list = [f"{col}-{method}" for col in aggs.keys() for method in aggs[col]]
df_agg.columns = new_col_list

df_agg = base.join(df_agg)

print(df_agg.shape)
display(df_agg.head())

utils.save_feature(df_feat=df_agg, is_train=2, target=target, ignore_list=[key, target], prefix=prefix)

(356255, 201)


,TARGET,AMT_CREDIT_per_AMT_ANNUITY-sum,AMT_CREDIT_per_AMT_ANNUITY-mean,AMT_CREDIT_per_AMT_ANNUITY-max,AMT_CREDIT_per_AMT_ANNUITY-min,AMT_CREDIT_per_AMT_ANNUITY-std,AMT_CREDIT_per_AMT_DOWN_PAYMENT-sum,AMT_CREDIT_per_AMT_DOWN_PAYMENT-mean,AMT_CREDIT_per_AMT_DOWN_PAYMENT-max,AMT_CREDIT_per_AMT_DOWN_PAYMENT-min,AMT_CREDIT_per_AMT_DOWN_PAYMENT-std,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-sum,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-mean,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-max,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-min,AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-std,AMT_APPLICATION_per_AMT_DOWN_PAYMENT-sum,AMT_APPLICATION_per_AMT_DOWN_PAYMENT-mean,AMT_APPLICATION_per_AMT_DOWN_PAYMENT-max,AMT_APPLICATION_per_AMT_DOWN_PAYMENT-min,AMT_APPLICATION_per_AMT_DOWN_PAYMENT-std,AMT_DOWN_PAYMENT_per_AMT_ANNUITY-sum,AMT_DOWN_PAYMENT_per_AMT_ANNUITY-mean,AMT_DOWN_PAYMENT_per_AMT_ANNUITY-max,AMT_DOWN_PAYMENT_per_AMT_ANNUITY-min,AMT_DOWN_PAYMENT_per_AMT_ANNUITY-std,TMP_INTEREST_RATE_APP-sum,TMP_INTEREST_RATE_APP-mean,TMP_INTEREST_RATE_APP-max,TMP_INTEREST_RATE_APP-min,TMP_INTEREST_RATE_APP-std,TMP_INTEREST_RATE_GOODS-sum,TMP_INTEREST_RATE_GOODS-mean,TMP_INTEREST_RATE_GOODS-max,TMP_INTEREST_RATE_GOODS-min,TMP_INTEREST_RATE_GOODS-std,AMT_INTEREST-sum,AMT_INTEREST-mean,AMT_INTEREST-max,AMT_INTEREST-min,AMT_INTEREST-std,AMT_INTEREST_GOODS-sum,AMT_INTEREST_GOODS-mean,AMT_INTEREST_GOODS-max,AMT_INTEREST_GOODS-min,AMT_INTEREST_GOODS-std,CNT_CREDIT_diff_CNT_APP-sum,CNT_CREDIT_diff_CNT_APP-mean,CNT_CREDIT_diff_CNT_APP-max,CNT_CREDIT_diff_CNT_APP-min,CNT_CREDIT_diff_CNT_APP-std,CNT_CREDIT_diff_CNT_GOODS-sum,CNT_CREDIT_diff_CNT_GOODS-mean,CNT_CREDIT_diff_CNT_GOODS-max,CNT_CREDIT_diff_CNT_GOODS-min,CNT_CREDIT_diff_CNT_GOODS-std,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN-sum,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN-mean,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN-max,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN-min,DOWN_PAYMENT_diff_CREDIT_pro_RATE_DOWN-std,FIRST_DRAW_TERM-sum,FIRST_DRAW_TERM-mean,FIRST_DRAW_TERM-max,FIRST_DRAW_TERM-min,FIRST_DRAW_TERM-std,FIRST_DRAW_DUE_TERM-sum,FIRST_DRAW_DUE_TERM-mean,FIRST_DRAW_DUE_TERM-max,FIRST_DRAW_DUE_TERM-min,FIRST_DRAW_DUE_TERM-std,FIRST_DUE_TERM-sum,FIRST_DUE_TERM-mean,FIRST_DUE_TERM-max,FIRST_DUE_TERM-min,FIRST_DUE_TERM-std,DAYS_LAST_DUE_diff_FIRST_DUE-sum,DAYS_LAST_DUE_diff_FIRST_DUE-mean,DAYS_LAST_DUE_diff_FIRST_DUE-max,DAYS_LAST_DUE_diff_FIRST_DUE-min,DAYS_LAST_DUE_diff_FIRST_DUE-std,DAYS_LAST_DUE_1st_diff_FIRST_DUE-sum,DAYS_LAST_DUE_1st_diff_FIRST_DUE-mean,DAYS_LAST_DUE_1st_diff_FIRST_DUE-max,DAYS_LAST_DUE_1st_diff_FIRST_DUE-min,DAYS_LAST_DUE_1st_diff_FIRST_DUE-std,DAYS_LAST_DUE_diff_DECISION-sum,DAYS_LAST_DUE_diff_DECISION-mean,DAYS_LAST_DUE_diff_DECISION-max,DAYS_LAST_DUE_diff_DECISION-min,DAYS_LAST_DUE_diff_DECISION-std,DAYS_LAST_DUE_1st_diff_DECISION-sum,DAYS_LAST_DUE_1st_diff_DECISION-mean,DAYS_LAST_DUE_1st_diff_DECISION-max,DAYS_LAST_DUE_1st_diff_DECISION-min,DAYS_LAST_DUE_1st_diff_DECISION-std,DAYS_TERMINATION_diff_DECISION-sum,DAYS_TERMINATION_diff_DECISION-mean,DAYS_TERMINATION_diff_DECISION-max,DAYS_TERMINATION_diff_DECISION-min,...,revo_AMT_CREDIT_per_revo_AMT_ANNUITY-sum,revo_AMT_CREDIT_per_revo_AMT_ANNUITY-mean,revo_AMT_CREDIT_per_revo_AMT_ANNUITY-max,revo_AMT_CREDIT_per_revo_AMT_ANNUITY-min,revo_AMT_CREDIT_per_revo_AMT_ANNUITY-std,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT-sum,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT-mean,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT-max,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT-min,revo_AMT_CREDIT_per_revo_AMT_DOWN_PAYMENT-std,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT-sum,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT-mean,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT-max,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT-min,revo_AMT_GOODS_PRICE_per_revo_AMT_DOWN_PAYMENT-std,revo_AMT_APPLICATION_per_revo_AMT_DOWN_PAYMENT-sum,revo_AMT_APPLICATION_per_revo_AMT_DOWN_PAYMENT-mean,revo_AMT_APPLICATION_per_revo_AMT_DOWN_PAYMENT-max,revo_AMT_APPLICATION_per_revo_AMT_DOWN_PAYMENT-min,revo_AMT_APPLICATION

(307511,) | AMT_CREDIT_per_AMT_ANNUITY-sum
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-mean
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-max
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-min
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-std
(307511,) | AMT_CREDIT_per_AMT_DOWN_PAYMENT-sum
(307511,) | AMT_CREDIT_per_AMT_DOWN_PAYMENT-mean
(307511,) | AMT_CREDIT_per_AMT_DOWN_PAYMENT-max
(307511,) | AMT_CREDIT_per_AMT_DOWN_PAYMENT-min
(307511,) | AMT_CREDIT_per_AMT_DOWN_PAYMENT-std
(307511,) | AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-sum
(307511,) | AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-mean
(307511,) | AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-max
(307511,) | AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-min
(307511,) | AMT_GOODS_PRICE_per_AMT_DOWN_PAYMENT-std
(307511,) | AMT_APPLICATION_per_AMT_DOWN_PAYMENT-sum
(307511,) | AMT_APPLICATION_per_AMT_DOWN_PAYMENT-mean
(307511,) | AMT_APPLICATION_per_AMT_DOWN_PAYMENT-max
(307511,) | AMT_APPLICATION_per_AMT_DOWN_PAYMENT-min
(307511,) | AMT_APPLICATION_per_AMT_DOWN_PAYMENT-std
(307511,) |

In [31]:
from new_FE import CategoricalEncoding
#========================================================================
# Feature Engineering 
cat_enc = CategoricalEncoding(data=pre, key=key, target=target, ignore_list=[key, target], base=base)
prefix = 'f002_pre_'
cat_list = get_categorical_features(pre, ignore_list)
df_ohe = cat_enc.go_ohe(cat_list=cat_list, method_list=['sum', 'mean'], prefix=prefix)
df_label = cat_enc.go_label(cat_list=cat_list, method_list=['mean', 'std', 'max', 'min'], prefix=prefix)
#========================================================================

2019-03-19 12:03:02,266 utils 346 [INFO]    [logger_func] start 
2019-03-19 12:03:02,266 utils 346 [INFO]    [logger_func] start 
2019-03-19 12:03:02,266 utils 346 [INFO]    [logger_func] start 
100%|██████████| 16/16 [00:15<00:00,  1.02s/it]


(307511,) | ohe_CHANNEL_TYPE_AP+ _Cash loan_-sum
(307511,) | ohe_CHANNEL_TYPE_AP+ _Cash loan_-mean
(307511,) | ohe_CHANNEL_TYPE_Car dealer-sum
(307511,) | ohe_CHANNEL_TYPE_Car dealer-mean
(307511,) | ohe_CHANNEL_TYPE_Channel of corporate sales-sum
(307511,) | ohe_CHANNEL_TYPE_Channel of corporate sales-mean
(307511,) | ohe_CHANNEL_TYPE_Contact center-sum
(307511,) | ohe_CHANNEL_TYPE_Contact center-mean
(307511,) | ohe_CHANNEL_TYPE_Country-wide-sum
(307511,) | ohe_CHANNEL_TYPE_Country-wide-mean
(307511,) | ohe_CHANNEL_TYPE_Credit and cash offices-sum
(307511,) | ohe_CHANNEL_TYPE_Credit and cash offices-mean
(307511,) | ohe_CHANNEL_TYPE_Regional _ Local-sum
(307511,) | ohe_CHANNEL_TYPE_Regional _ Local-mean
(307511,) | ohe_CHANNEL_TYPE_Stone-sum
(307511,) | ohe_CHANNEL_TYPE_Stone-mean
(307511,) | ohe_CODE_REJECT_REASON_CLIENT-sum
(307511,) | ohe_CODE_REJECT_REASON_CLIENT-mean
(307511,) | ohe_CODE_REJECT_REASON_HC-sum
(307511,) | ohe_CODE_REJECT_REASON_HC-mean
(307511,) | ohe_CODE_REJECT_

 12%|█▎        | 2/16 [00:00<00:00, 15.58it/s]

[One Hot Encoding] done in 48 s


100%|██████████| 16/16 [00:00<00:00, 16.49it/s]


[Factorize Encoding] done in 1 s
[Aggregation] done in 3 s
(307511,) | label_CHANNEL_TYPE-mean
(307511,) | label_CHANNEL_TYPE-std
(307511,) | label_CHANNEL_TYPE-max
(307511,) | label_CHANNEL_TYPE-min
(307511,) | label_CODE_REJECT_REASON-mean
(307511,) | label_CODE_REJECT_REASON-std
(307511,) | label_CODE_REJECT_REASON-max
(307511,) | label_CODE_REJECT_REASON-min
(307511,) | label_FLAG_LAST_APPL_PER_CONTRACT-mean
(307511,) | label_FLAG_LAST_APPL_PER_CONTRACT-std
(307511,) | label_FLAG_LAST_APPL_PER_CONTRACT-max
(307511,) | label_FLAG_LAST_APPL_PER_CONTRACT-min
(307511,) | label_NAME_CASH_LOAN_PURPOSE-mean
(307511,) | label_NAME_CASH_LOAN_PURPOSE-std
(307511,) | label_NAME_CASH_LOAN_PURPOSE-max
(307511,) | label_NAME_CASH_LOAN_PURPOSE-min
(307511,) | label_NAME_CLIENT_TYPE-mean
(307511,) | label_NAME_CLIENT_TYPE-std
(307511,) | label_NAME_CLIENT_TYPE-max
(307511,) | label_NAME_CLIENT_TYPE-min
(307511,) | label_NAME_CONTRACT_STATUS-mean
(307511,) | label_NAME_CONTRACT_STATUS-std
(307511,)